In [2]:
import pandas as pd
import numpy as np

# Read in data file from bucket
df = pd.read_csv('gs://case3-19mar23/fh_5yrs.csv')

# Create list of symbols
all_unique_symbols = df['symbol'].unique()
#symbols = ['SPPI', 'AACG']

#Q6 - Choose stocks to buy and hold - stocks are chosen at random
pick_six = pd.Series(all_unique_symbols).sample(n = 6)
symbols = pick_six
#symbols = all_unique_symbols
#print("\Randomly chosen stocks:\n", symbols)


# Define the time periods and percent changes
time_periods = ['1wk', '2wk', '3wk', '1mo', '2mo', '3mo', '4mo', '5mo', '6mo']
percent_levels = [.01, .02, .03, .04, .05, .10, .15, .20]


# Create an empty dictionary to store the matrices for each symbol (one for each of golden/death cross)
gc_matrix_dict = {}
dc_matrix_dict = {}

# Loop through symbols to create a matrix for each symbol
for symbol in symbols:
    # Read the historical data from the CSV file into a DataFrame
    df = pd.read_csv('gs://case3-19mar23/fh_5yrs.csv', index_col="date", parse_dates=True)
    df = df[df["symbol"] == symbol]

    # Calculate the 50-day and 200-day moving averages
    df["ma50"] = df["adjclose"].rolling(window=50).mean()
    df["ma200"] = df["adjclose"].rolling(window=200).mean()
    
    df = df.dropna()

    #Q1 - Find all golden crosses for each stock
    # Calculate golden cross
    df["golden_cross"] = np.where(df["ma50"] > df["ma200"], True, False)

    #Q3 - Find all death crosses for each stock
    # Calculate death cross
    df["death_cross"] = np.where(df["ma50"] < df["ma200"], True, False)
    
    # Create an empty matrix with the time intervals as columns and percent changes as rows
    gc_matrix = pd.DataFrame(columns=time_periods, index=percent_levels)
    dc_matrix = pd.DataFrame(columns=time_periods, index=percent_levels)

    # Calculate the percent increase/decrease for each time period and percent increase/decrease after a golden cross or death cross occurs
    for period in time_periods:
        if period == '1wk':
            period_length = 5
        elif period == '2wk':
            period_length = 10
        elif period == '3wk':
            period_length = 15
        elif period == '1mo':
            period_length = 20
        elif period == '2mo':
            period_length = 40
        elif period == '3mo':
            period_length = 60
        elif period == '4mo':
            period_length = 80
        elif period == '5mo':
            period_length = 100
        else:
            period_length = 120
            
        for percent in percent_levels:
            #Q2 & Q4 - What percentage of golden/death crosses result in a price increase/decrease by indicated percentages?
            # Calculate the percent change after a golden/death cross occurs
            df["pct_change"] = df["adjclose"].pct_change(periods=period_length)
            gc_matrix.loc[percent, period] = round(df.loc[df["golden_cross"], "pct_change"].quantile(percent), 2)
            dc_matrix.loc[percent, period] = round(df.loc[df["death_cross"], "pct_change"].quantile(percent), 2)

    # Store the matrices in respective dictionary
    gc_matrix_dict[symbol] = gc_matrix
    dc_matrix_dict[symbol] = dc_matrix

# Display the golden cross matrices for each symbol
for symbol, gc_matrix in gc_matrix_dict.items():
    print(f"Golden Cross Changes: {symbol}\n{gc_matrix}\n")

    # Display the death cross matrices for each symbol
for symbol, gc_matrix in gc_matrix_dict.items():
    print(f"Death Cross Changes: {symbol}\n{dc_matrix}\n")

Golden Cross Changes: JWN
       1wk   2wk   3wk   1mo   2mo   3mo   4mo   5mo   6mo
0.01 -0.15 -0.18 -0.21 -0.21  -0.2 -0.22 -0.24 -0.16 -0.04
0.02 -0.12 -0.16 -0.18  -0.2 -0.19 -0.21 -0.21 -0.12 -0.03
0.03  -0.1 -0.15 -0.16 -0.18 -0.18  -0.2 -0.18 -0.08 -0.02
0.04 -0.09 -0.14 -0.16 -0.16 -0.18 -0.17 -0.16 -0.04 -0.02
0.05 -0.07 -0.12 -0.14 -0.14 -0.17 -0.16 -0.15 -0.03 -0.01
0.10 -0.05 -0.08 -0.08  -0.1 -0.14 -0.13 -0.06  0.01  0.03
0.15 -0.04 -0.05 -0.06 -0.07 -0.08  -0.1 -0.02  0.03  0.04
0.20 -0.03 -0.04 -0.04 -0.05 -0.05 -0.04   0.0  0.05  0.07

Golden Cross Changes: CBLI
       1wk   2wk   3wk   1mo   2mo   3mo   4mo   5mo   6mo
0.01 -0.41 -0.57 -0.56 -0.54 -0.57 -0.47  -0.4 -0.42 -0.52
0.02 -0.25  -0.3 -0.42 -0.42 -0.42 -0.42 -0.38 -0.39 -0.46
0.03 -0.18 -0.27 -0.31 -0.37 -0.34 -0.32 -0.31 -0.23 -0.22
0.04 -0.16 -0.23 -0.28 -0.32 -0.33 -0.26 -0.23 -0.17 -0.16
0.05 -0.14 -0.19 -0.26 -0.29 -0.31 -0.24 -0.21 -0.15 -0.13
0.10 -0.09 -0.15 -0.17 -0.19  -0.2 -0.17 -0.11 -0.09 -0.04
0.